In [1]:
import numpy as np
import librosa
import noisereduce as nr
import matplotlib.pyplot as plt
import sounddevice as sd
import IPython.display as ipd
import pandas as pd
from librosa import effects
import scipy.signal as sp
import statistics as st

In [2]:
def get_audio_split(y,sr):
    duration_ogn = len(y)/sr
    if(duration_ogn <= 30.0):

      data = 15 * sr
    elif(duration_ogn <= 15.0):
      data = 10 * sr 
    elif(duration_ogn <= 10.0):
      data = 5 * sr
    else:
      data = 30 * sr   

    return data  


In [3]:
def get_snip(y,sr):
  data = get_audio_split(y,sr)
  y = y[:data]
  return y

In [4]:
def get_n_fft_val(y,sr):
  duration = len(y)/sr

  if(duration < 2.0):
    n_fft_val =512
  else:
    n_fft_val = 1024   

  return n_fft_val

In [5]:
def reduce_audio_noise(y,sr,n_fft_val):

  reduced_noise = nr.reduce_noise(y=y,sr=sr,stationary=False,prop_decrease=0.8,n_fft=n_fft_val,freq_mask_smooth_hz=300, time_mask_smooth_ms=50)  

  return reduced_noise

In [6]:
def get_compress(reduced_noise):
  
  y_comp = librosa.effects.preemphasis(reduced_noise, coef=0.97)

  return y_comp


In [7]:
def get_s(y):
    S,phase = librosa.magphase(librosa.stft(y))
    return S

In [8]:
def get_rms(y):
    rms = librosa.feature.rms(S=get_s(y))
    return rms

In [9]:
def get_avg_rms(y):
    rms = get_rms(y)
    avg = np.mean(rms)
    return avg


In [10]:
def get_freq(y,sr):
  f0 = librosa.yin(y, fmin=50, fmax=10000,sr=sr)
  avg_fun_freq = np.mean(f0)
  return avg_fun_freq

In [11]:
def get_spect_cent(y):
  spect_cent = librosa.feature.spectral_centroid(S=get_s(y))
  return np.mean(spect_cent)

In [12]:
def get_snr(y_clean,y):
    
    signal_power = np.mean(y_clean ** 2)
    noise_power = np.mean(y ** 2)
    
    if noise_power == 0:
        return float('inf')
    
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

In [13]:
def get_harmonic(y):
    harmonic = librosa.effects.harmonic(y)
    return harmonic

In [14]:
def get_hnr_high(y, sr):
    f0 = librosa.yin(y, fmin=50, fmax=10000, sr=sr) 
    harmonic = librosa.effects.harmonic(y, margin=8)
    noise = y - harmonic                              
    hnr_high = 10 * np.log10(np.var(harmonic) / np.var(noise))
    return hnr_high

In [15]:
def get_hnr_low(y, sr):
    f0 = librosa.yin(y, fmin=50, fmax=10000, sr=sr) 
    harmonic = librosa.effects.harmonic(y, margin=1)
    noise = y - harmonic                              
    hnr_high = 10 * np.log10(np.var(harmonic) / np.var(noise))
    return hnr_high

In [16]:
def get_tonnetz(y,sr):
    tonnetz = librosa.feature.tonnetz(y=get_harmonic(y), sr=sr)
    return np.mean(tonnetz)

In [17]:
def get_mfcc(y,sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    return np.mean(mfcc)

In [18]:
def get_roll_off(y,sr):
    roll_off = librosa.feature.spectral_rolloff(S=get_s(y), sr=sr)
    return np.mean(roll_off)

In [19]:
def get_flatness(y):
    flatness = librosa.feature.spectral_flatness(S=get_s(y))
    return np.mean(flatness)

In [20]:
def get_contrast(y,sr):
    contrast = librosa.feature.spectral_contrast(S=get_s(y), sr=sr)
    return np.mean(contrast)

In [21]:
df = pd.read_csv('random_train.csv')

In [22]:
df.head(10)

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,class_name
0,523060,[''],[''],523060/CSA34181.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,-3.8333,-70.3333,Eschatoceras bipunctatus,Eschatoceras bipunctatus,Fernando Montealegre-Z,cc-by-nc-sa 4.0,Insecta
1,715170,[''],[''],715170/CSA35806.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.6519,-73.5077,Copris susanae,Copris susanae,Angela M. Mendoza-Henao,cc-by-nc-sa 4.0,Insecta
2,48124,[''],[''],48124/CSA18785.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Tettigoniidae,Katydids,Diego A Gómez-Morales,cc-by-nc-sa 4.0,Insecta
3,1564122,[''],[''],1564122/CSA34200.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,-3.8333,-70.3333,Daedadelus waehnerorum,Daedadelus waehnerorum,Fernando Montealegre-Z,cc-by-nc-sa 4.0,Insecta
4,52884,[''],[''],52884/CSA36344.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Gryllidae,True Crickets,Fabio A. Sarria-S,cc-by-nc-sa 4.0,Insecta
5,50186,[''],[''],50186/CSA18282.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,6.4365,-74.1317,Cicadidae,Typical Cicadas,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,Insecta
6,52884,[''],[''],52884/CSA18786.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Gryllidae,True Crickets,Diego A Gómez-Morales,cc-by-nc-sa 4.0,Insecta
7,868458,[''],[''],868458/CSA34218.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,-3.8333,-70.3333,Typophyllum inflatum,Typophyllum inflatum,Fernando Montealegre-Z,cc-by-nc-sa 4.0,Insecta
8,963335,[''],[''],963335/CSA36377.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Subria sylvestris,Subria sylvestris,Fabio A. Sarria-S,cc-by-nc-sa 4.0,Insecta
9,50186,[''],[''],50186/CSA34622.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,3.8872,-73.5909,Cicadidae,Typical Cicadas,Eliana Barona- Cortés,cc-by-nc-sa 4.0,Insecta


In [ ]:
for i in range(0, len(df)):
    y,sr = librosa.load(f'{'train_audio/'}{df.filename[i]}')
   # print(i)
    if(df.collection[i] == 'CSA'):
        y = get_snip(y,sr)

    n_fft = get_n_fft_val(y,sr)
    y_red = reduce_audio_noise(y,sr,n_fft)
    y_comp = get_compress(y_red)

    df.loc[i,['rms_reduced']] = get_avg_rms(y_red) 
    df.loc[i,['rms_compressed']] = get_avg_rms(y_comp)

    df.loc[i,['frequency_reduced']] = get_freq(y_red,sr)
    df.loc[i,['frequency_compressed']] = get_freq(y_comp,sr)

    df.loc[i,['spectral_centroid_reduced']] = get_spect_cent(y_red)
    df.loc[i,['spectral_centroid_compressed']] = get_spect_cent(y_comp)
    
    df.loc[i,['snr_reduced']] = get_snr(y_red,y)
    df.loc[i,['snr_compressed']] = get_snr(y_comp,y)

    df.loc[i,['high_hnr_reduced']] = get_hnr_high(y_red,sr)
    df.loc[i,['low_hnr_reduced']] = get_hnr_low(y_red,sr)

    df.loc[i,['high_hnr_compressed']] = get_hnr_high(y_comp,sr)
    df.loc[i,['low_hnr_compressed']] = get_hnr_low(y_comp,sr)

    df.loc[i,['tonnetz_reduced']] = get_tonnetz(y_red,sr)
    df.loc[i,['tonnetz_compressed']] = get_tonnetz(y_comp,sr)

    df.loc[i,['mfcc_reduced']] = get_mfcc(y_red,sr)
    df.loc[i,['mfcc_compressed']] = get_mfcc(y_comp,sr)

    df.loc[i,['roll_off_reduced']] = get_roll_off(y_red,sr)
    df.loc[i,['roll_off_compressed']] = get_roll_off(y_comp,sr)

    df.loc[i,['flatness_reduced']] = get_flatness(y_red)
    df.loc[i,['flatness_compressed']] = get_flatness(y_comp)

    df.loc[i,['contrast_reduced']] = get_contrast(y_red,sr)
    df.loc[i,['contrast_compressed']] = get_contrast(y_comp,sr)
    

print("done.")   

In [ ]:
df.to_csv('new_train.csv', index=False)